In [5]:
from __future__ import print_function, division

import plaidml.keras
plaidml.keras.install_backend()

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise, Lambda
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, merge
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np


def build_encoder():
    # Encoder

    img = Input(shape=img_shape)

    h = Flatten()(img)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    mu = Dense(latent_dim)(h)
    log_var = Dense(latent_dim)(h)
    latent_repr = Lambda(lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2), lambda p: p[0])([mu, log_var])

    return Model(img, latent_repr)

def build_decoder():

    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    z = Input(shape=(latent_dim,))
    img = model(z)

    return Model(z, img)

def build_discriminator():

    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation="sigmoid"))
    model.summary()

    encoded_repr = Input(shape=(latent_dim, ))
    validity = model(encoded_repr)

    return Model(encoded_repr, validity)

def train(epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        latent_fake = encoder.predict(imgs)
        latent_real = np.random.normal(size=(batch_size, latent_dim))

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(latent_real, valid)
        d_loss_fake = discriminator.train_on_batch(latent_fake, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator
        g_loss = adversarial_autoencoder.train_on_batch(imgs, [imgs, valid])

        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images(epoch)

def sample_images(epoch):
    r, c = 5, 5

    z = np.random.normal(size=(r*c, latent_dim))
    gen_imgs = decoder.predict(z)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

def save_model():

    def save(model, model_name):
        model_path = "saved_model/%s.json" % model_name
        weights_path = "saved_model/%s_weights.hdf5" % model_name
        options = {"file_arch": model_path,
                    "file_weight": weights_path}
        json_string = model.to_json()
        open(options['file_arch'], 'w').write(json_string)
        model.save_weights(options['file_weight'])

    save(generator, "aae_generator")
    save(discriminator, "aae_discriminator")



In [6]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 10

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the encoder / decoder
encoder = build_encoder()
decoder = build_decoder()

img = Input(shape=img_shape)
# The generator takes the image, encodes it and reconstructs it
# from the encoding
encoded_repr = encoder(img)
reconstructed_img = decoder(encoded_repr)

# For the adversarial_autoencoder model we will only train the generator
discriminator.trainable = False

# The discriminator determines validity of the encoding
validity = discriminator(encoded_repr)

# The adversarial_autoencoder model  (stacked generator and discriminator)
adversarial_autoencoder = Model(img, [reconstructed_img, validity])
adversarial_autoencoder.compile(loss=['mse', 'binary_crossentropy'],
    loss_weights=[0.999, 0.001],
    optimizer=optimizer)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 137,217
Trainable params: 137,217
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dens

In [ ]:
train(epochs=20000, batch_size=32, sample_interval=200)

C:\Users\micro\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.713208, acc: 43.75%] [G loss: 0.944666, mse: 0.944890]
1 [D loss: 0.785441, acc: 35.94%] [G loss: 0.927547, mse: 0.927900]
2 [D loss: 0.893810, acc: 28.12%] [G loss: 0.913179, mse: 0.913599]
3 [D loss: 0.913595, acc: 34.38%] [G loss: 0.884751, mse: 0.885123]
4 [D loss: 0.811383, acc: 28.12%] [G loss: 0.829984, mse: 0.830050]
5 [D loss: 0.635220, acc: 82.81%] [G loss: 0.764438, mse: 0.764036]
6 [D loss: 0.564779, acc: 73.44%] [G loss: 0.718324, mse: 0.717661]
7 [D loss: 0.595824, acc: 76.56%] [G loss: 0.703671, mse: 0.703141]
8 [D loss: 0.708687, acc: 48.44%] [G loss: 0.706621, mse: 0.706384]
9 [D loss: 0.771485, acc: 34.38%] [G loss: 0.693323, mse: 0.693162]
10 [D loss: 0.901698, acc: 26.56%] [G loss: 0.691639, mse: 0.691510]
11 [D loss: 0.832210, acc: 32.81%] [G loss: 0.671268, mse: 0.670978]
12 [D loss: 0.789130, acc: 28.12%] [G loss: 0.670539, mse: 0.670076]
13 [D loss: 0.880614, acc: 26.56%] [G loss: 0.689012, mse: 0.688593]
14 [D loss: 0.792937, acc: 32.81%] [G loss: 

119 [D loss: 0.719027, acc: 25.00%] [G loss: 0.269190, mse: 0.268755]
120 [D loss: 0.725882, acc: 29.69%] [G loss: 0.272057, mse: 0.271611]
121 [D loss: 0.701634, acc: 43.75%] [G loss: 0.274301, mse: 0.273844]
122 [D loss: 0.701888, acc: 50.00%] [G loss: 0.284054, mse: 0.283603]
123 [D loss: 0.688767, acc: 50.00%] [G loss: 0.273317, mse: 0.272886]
124 [D loss: 0.727798, acc: 34.38%] [G loss: 0.288726, mse: 0.288348]
125 [D loss: 0.751231, acc: 20.31%] [G loss: 0.292560, mse: 0.292219]
126 [D loss: 0.746640, acc: 20.31%] [G loss: 0.276811, mse: 0.276421]
127 [D loss: 0.709383, acc: 28.12%] [G loss: 0.264902, mse: 0.264413]
128 [D loss: 0.665736, acc: 71.88%] [G loss: 0.267001, mse: 0.266448]
129 [D loss: 0.650783, acc: 75.00%] [G loss: 0.270982, mse: 0.270435]
130 [D loss: 0.667958, acc: 67.19%] [G loss: 0.280721, mse: 0.280210]
131 [D loss: 0.708684, acc: 51.56%] [G loss: 0.312056, mse: 0.311671]
132 [D loss: 0.738012, acc: 34.38%] [G loss: 0.258061, mse: 0.257666]
133 [D loss: 0.75404

237 [D loss: 0.667034, acc: 75.00%] [G loss: 0.236861, mse: 0.236230]
238 [D loss: 0.689240, acc: 65.62%] [G loss: 0.254879, mse: 0.254331]
239 [D loss: 0.691617, acc: 51.56%] [G loss: 0.257056, mse: 0.256581]
240 [D loss: 0.715267, acc: 32.81%] [G loss: 0.246046, mse: 0.245629]
241 [D loss: 0.757466, acc: 17.19%] [G loss: 0.243933, mse: 0.243551]
242 [D loss: 0.789067, acc: 20.31%] [G loss: 0.267233, mse: 0.266895]
243 [D loss: 0.801990, acc: 17.19%] [G loss: 0.270038, mse: 0.269718]
244 [D loss: 0.768906, acc: 25.00%] [G loss: 0.294056, mse: 0.293727]
245 [D loss: 0.755248, acc: 17.19%] [G loss: 0.267932, mse: 0.267499]
246 [D loss: 0.707053, acc: 42.19%] [G loss: 0.267120, mse: 0.266604]
247 [D loss: 0.691442, acc: 60.94%] [G loss: 0.295235, mse: 0.294723]
248 [D loss: 0.692122, acc: 51.56%] [G loss: 0.259461, mse: 0.258917]
249 [D loss: 0.709197, acc: 43.75%] [G loss: 0.274759, mse: 0.274268]
250 [D loss: 0.713138, acc: 37.50%] [G loss: 0.262879, mse: 0.262408]
251 [D loss: 0.71187

355 [D loss: 0.727136, acc: 15.62%] [G loss: 0.234189, mse: 0.233724]
356 [D loss: 0.713168, acc: 37.50%] [G loss: 0.226010, mse: 0.225522]
357 [D loss: 0.707538, acc: 43.75%] [G loss: 0.240778, mse: 0.240309]
358 [D loss: 0.709948, acc: 37.50%] [G loss: 0.242079, mse: 0.241614]
359 [D loss: 0.718826, acc: 26.56%] [G loss: 0.236649, mse: 0.236199]
360 [D loss: 0.718136, acc: 23.44%] [G loss: 0.254445, mse: 0.254024]
361 [D loss: 0.717049, acc: 25.00%] [G loss: 0.239157, mse: 0.238712]
362 [D loss: 0.707349, acc: 39.06%] [G loss: 0.259744, mse: 0.259309]
363 [D loss: 0.701380, acc: 39.06%] [G loss: 0.270305, mse: 0.269849]
364 [D loss: 0.681957, acc: 65.62%] [G loss: 0.246372, mse: 0.245872]
365 [D loss: 0.665231, acc: 79.69%] [G loss: 0.244781, mse: 0.244263]
366 [D loss: 0.665748, acc: 73.44%] [G loss: 0.244151, mse: 0.243626]
367 [D loss: 0.672964, acc: 65.62%] [G loss: 0.263630, mse: 0.263161]
368 [D loss: 0.683462, acc: 51.56%] [G loss: 0.243879, mse: 0.243431]
369 [D loss: 0.70899

...

8798 [D loss: 0.695612, acc: 40.62%] [G loss: 0.096680, mse: 0.096082]
8799 [D loss: 0.691540, acc: 56.25%] [G loss: 0.083070, mse: 0.082454]
8800 [D loss: 0.694235, acc: 51.56%] [G loss: 0.090295, mse: 0.089688]
8801 [D loss: 0.691854, acc: 53.12%] [G loss: 0.082421, mse: 0.081806]
8802 [D loss: 0.691808, acc: 48.44%] [G loss: 0.086714, mse: 0.086104]
8803 [D loss: 0.698701, acc: 35.94%] [G loss: 0.087575, mse: 0.086970]
8804 [D loss: 0.694004, acc: 59.38%] [G loss: 0.082258, mse: 0.081643]
8805 [D loss: 0.689191, acc: 54.69%] [G loss: 0.081111, mse: 0.080482]
8806 [D loss: 0.691932, acc: 56.25%] [G loss: 0.082292, mse: 0.081673]
8807 [D loss: 0.693872, acc: 48.44%] [G loss: 0.080164, mse: 0.079545]
8808 [D loss: 0.693319, acc: 48.44%] [G loss: 0.078447, mse: 0.077823]
8809 [D loss: 0.687339, acc: 65.62%] [G loss: 0.078107, mse: 0.077476]
8810 [D loss: 0.692188, acc: 48.44%] [G loss: 0.085374, mse: 0.084755]
8811 [D loss: 0.696778, acc: 43.75%] [G loss: 0.099074, mse: 0.098480]
8812 [

8915 [D loss: 0.690045, acc: 60.94%] [G loss: 0.075796, mse: 0.075168]
8916 [D loss: 0.694406, acc: 50.00%] [G loss: 0.096405, mse: 0.095806]
8917 [D loss: 0.697716, acc: 43.75%] [G loss: 0.093996, mse: 0.093398]
8918 [D loss: 0.699048, acc: 37.50%] [G loss: 0.086351, mse: 0.085743]
8919 [D loss: 0.694222, acc: 48.44%] [G loss: 0.086343, mse: 0.085733]
8920 [D loss: 0.693987, acc: 53.12%] [G loss: 0.075878, mse: 0.075256]
8921 [D loss: 0.689874, acc: 60.94%] [G loss: 0.069571, mse: 0.068937]
8922 [D loss: 0.692285, acc: 53.12%] [G loss: 0.099170, mse: 0.098569]
8923 [D loss: 0.698143, acc: 32.81%] [G loss: 0.076287, mse: 0.075671]
8924 [D loss: 0.691699, acc: 51.56%] [G loss: 0.080142, mse: 0.079522]
8925 [D loss: 0.690186, acc: 57.81%] [G loss: 0.084318, mse: 0.083701]
8926 [D loss: 0.696260, acc: 42.19%] [G loss: 0.082369, mse: 0.081753]
8927 [D loss: 0.691281, acc: 56.25%] [G loss: 0.072817, mse: 0.072188]
8928 [D loss: 0.695333, acc: 42.19%] [G loss: 0.069728, mse: 0.069098]
8929 [

9034 [D loss: 0.693083, acc: 50.00%] [G loss: 0.075695, mse: 0.075074]
9035 [D loss: 0.699256, acc: 28.12%] [G loss: 0.086253, mse: 0.085647]
9036 [D loss: 0.689522, acc: 62.50%] [G loss: 0.076625, mse: 0.076000]
9037 [D loss: 0.694141, acc: 50.00%] [G loss: 0.079724, mse: 0.079104]
9038 [D loss: 0.692843, acc: 53.12%] [G loss: 0.077020, mse: 0.076398]
9039 [D loss: 0.690885, acc: 54.69%] [G loss: 0.075485, mse: 0.074863]
9040 [D loss: 0.693838, acc: 56.25%] [G loss: 0.079630, mse: 0.079011]
9041 [D loss: 0.693389, acc: 46.88%] [G loss: 0.096031, mse: 0.095426]
9042 [D loss: 0.692397, acc: 48.44%] [G loss: 0.072448, mse: 0.071823]
9043 [D loss: 0.692111, acc: 51.56%] [G loss: 0.079431, mse: 0.078810]
9044 [D loss: 0.692237, acc: 53.12%] [G loss: 0.082963, mse: 0.082351]
9045 [D loss: 0.694536, acc: 42.19%] [G loss: 0.090747, mse: 0.090142]
9046 [D loss: 0.693464, acc: 53.12%] [G loss: 0.090464, mse: 0.089861]
9047 [D loss: 0.693459, acc: 53.12%] [G loss: 0.079704, mse: 0.079089]
9048 [

9152 [D loss: 0.694252, acc: 50.00%] [G loss: 0.099256, mse: 0.098653]
9153 [D loss: 0.693071, acc: 60.94%] [G loss: 0.078475, mse: 0.077854]
9154 [D loss: 0.694446, acc: 51.56%] [G loss: 0.066082, mse: 0.065444]
9155 [D loss: 0.691181, acc: 59.38%] [G loss: 0.082840, mse: 0.082218]
9156 [D loss: 0.693305, acc: 50.00%] [G loss: 0.088473, mse: 0.087859]
9157 [D loss: 0.694676, acc: 43.75%] [G loss: 0.082808, mse: 0.082195]
9158 [D loss: 0.695392, acc: 46.88%] [G loss: 0.078844, mse: 0.078227]
9159 [D loss: 0.696531, acc: 42.19%] [G loss: 0.073748, mse: 0.073121]
9160 [D loss: 0.693595, acc: 53.12%] [G loss: 0.084729, mse: 0.084114]
9161 [D loss: 0.692432, acc: 60.94%] [G loss: 0.076788, mse: 0.076162]
9162 [D loss: 0.690685, acc: 65.62%] [G loss: 0.075693, mse: 0.075066]
9163 [D loss: 0.693215, acc: 50.00%] [G loss: 0.076112, mse: 0.075490]
9164 [D loss: 0.693677, acc: 50.00%] [G loss: 0.068777, mse: 0.068148]
9165 [D loss: 0.693327, acc: 56.25%] [G loss: 0.085395, mse: 0.084780]
9166 [

9269 [D loss: 0.693817, acc: 43.75%] [G loss: 0.084372, mse: 0.083760]
9270 [D loss: 0.694519, acc: 51.56%] [G loss: 0.089144, mse: 0.088532]
9271 [D loss: 0.695973, acc: 48.44%] [G loss: 0.077573, mse: 0.076959]
9272 [D loss: 0.696884, acc: 46.88%] [G loss: 0.073595, mse: 0.072979]
9273 [D loss: 0.695792, acc: 50.00%] [G loss: 0.090053, mse: 0.089449]
9274 [D loss: 0.690057, acc: 65.62%] [G loss: 0.080781, mse: 0.080163]
9275 [D loss: 0.691743, acc: 51.56%] [G loss: 0.070519, mse: 0.069893]
9276 [D loss: 0.694569, acc: 54.69%] [G loss: 0.079352, mse: 0.078737]
9277 [D loss: 0.693071, acc: 57.81%] [G loss: 0.084714, mse: 0.084103]
9278 [D loss: 0.695910, acc: 40.62%] [G loss: 0.084178, mse: 0.083569]
9279 [D loss: 0.689224, acc: 54.69%] [G loss: 0.081375, mse: 0.080759]
9280 [D loss: 0.694604, acc: 43.75%] [G loss: 0.090528, mse: 0.089924]
9281 [D loss: 0.690951, acc: 59.38%] [G loss: 0.074261, mse: 0.073636]
9282 [D loss: 0.693600, acc: 53.12%] [G loss: 0.094523, mse: 0.093922]
9283 [